In [1]:
from collections import Counter

from pyspark import SparkConf, SparkContext
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, count, isnan, when

display_opt = SparkConf().set("spark.sql.repl.eagerEval.enabled", True)
sc = SparkContext(conf=display_opt)
sqlContext = SQLContext(sc)

24/03/21 20:42:54 WARN Utils: Your hostname, carl-Precision-7780 resolves to a loopback address: 127.0.1.1; using 192.168.1.164 instead (on interface enp0s31f6)
24/03/21 20:42:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/21 20:42:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/carl/anaconda3/envs/p8env/lib/python3.12/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
# load csv in spark
df = (
    sqlContext.read.format("com.databricks.spark.csv")
    .options(header="true", inferschema="true")
    .load("../data/raw/2016_Building_Energy_Benchmarking.csv")
)
# df.take(1)

In [3]:
# print columns
df.cache()
df.printSchema()

root
 |-- OSEBuildingID: integer (nullable = true)
 |-- DataYear: integer (nullable = true)
 |-- BuildingType: string (nullable = true)
 |-- PrimaryPropertyType: string (nullable = true)
 |-- PropertyName: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- ZipCode: integer (nullable = true)
 |-- TaxParcelIdentificationNumber: string (nullable = true)
 |-- CouncilDistrictCode: integer (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- NumberofBuildings: integer (nullable = true)
 |-- NumberofFloors: integer (nullable = true)
 |-- PropertyGFATotal: integer (nullable = true)
 |-- PropertyGFAParking: integer (nullable = true)
 |-- PropertyGFABuilding(s): integer (nullable = true)
 |-- ListOfAllPropertyUseTypes: string (nullable = true)
 |-- LargestPropertyUseType:

In [4]:
# spark describe
df.describe().toPandas()

,summary,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,...,SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
0,count,3376,3376,3376,3376,3376,3376,3376,3376,3360,...,3367,3367,3367,3367,3367,0,3376,32,3367,3367
1,mean,21208.991113744076,2016.0,None,None,3354.230769230769,None,None,None,98116.94910714286,...,274595.8982093198,1086638.9665709417,3707612.1615942973,13685.04537626648,1368504.5414427682,None,None,None,119.72397089397079,1.1759162459162464
2,stddev,12223.7570149853,0.0,None,None,2224.754299312104,None,None,None,18.61520454431558,...,3912173.392696105,4352478.355209237,1.4850656138963334E7,67097.80829551257,6709780.834880091,None,None,None,538.8322265068628,1.821451787910054
3,min,1,2016,Campus,Distribution Center,#4706 Bitterlake,(ID23682) 3204 SW MORGAN ST,Seattle,WA,98006,...,0.0,-33826.80078,-115417.0,0.0,0.0,None,Compliant,High outlier,-0.8,-0.02
4,max,50226,2016,SPS-District K-12,Worship Facility,westwinds,Readiness Center - Pier 91,Seattle,WA,98272,...,1.34943456E8,1.92577488E8,6.57074389E8,2979090.0,2.97909E8,None,Non-Compliant,Low outlier,16870.98,34.09


In [5]:
df.dtypes

[('OSEBuildingID', 'int'),
 ('DataYear', 'int'),
 ('BuildingType', 'string'),
 ('PrimaryPropertyType', 'string'),
 ('PropertyName', 'string'),
 ('Address', 'string'),
 ('City', 'string'),
 ('State', 'string'),
 ('ZipCode', 'int'),
 ('TaxParcelIdentificationNumber', 'string'),
 ('CouncilDistrictCode', 'int'),
 ('Neighborhood', 'string'),
 ('Latitude', 'double'),
 ('Longitude', 'double'),
 ('YearBuilt', 'int'),
 ('NumberofBuildings', 'int'),
 ('NumberofFloors', 'int'),
 ('PropertyGFATotal', 'int'),
 ('PropertyGFAParking', 'int'),
 ('PropertyGFABuilding(s)', 'int'),
 ('ListOfAllPropertyUseTypes', 'string'),
 ('LargestPropertyUseType', 'string'),
 ('LargestPropertyUseTypeGFA', 'int'),
 ('SecondLargestPropertyUseType', 'string'),
 ('SecondLargestPropertyUseTypeGFA', 'double'),
 ('ThirdLargestPropertyUseType', 'string'),
 ('ThirdLargestPropertyUseTypeGFA', 'double'),
 ('YearsENERGYSTARCertified', 'double'),
 ('ENERGYSTARScore', 'string'),
 ('SiteEUI(kBtu/sf)', 'double'),
 ('SiteEUIWN(kBtu/sf

In [6]:
# select numerical features
# TODO: categorical and boolean features
numeric_features = [c[0] for c in df.dtypes if c[1] == "int" or c[1] == "double"]

In [7]:
### Get count of nan or missing values in pyspark
df.select([count(when(isnan(c), c)).alias(c) for c in numeric_features]).show()

24/03/21 20:43:10 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+-------------+--------+-------+-------------------+--------+---------+---------+-----------------+--------------+----------------+------------------+----------------------+-------------------------+-------------------------------+------------------------------+------------------------+----------------+------------------+------------------+--------------------+-------------------+---------------------+--------------+----------------+-----------------+------------------+----------------+-----------------+---------------------+
|OSEBuildingID|DataYear|ZipCode|CouncilDistrictCode|Latitude|Longitude|YearBuilt|NumberofBuildings|NumberofFloors|PropertyGFATotal|PropertyGFAParking|PropertyGFABuilding(s)|LargestPropertyUseTypeGFA|SecondLargestPropertyUseTypeGFA|ThirdLargestPropertyUseTypeGFA|YearsENERGYSTARCertified|SiteEUI(kBtu/sf)|SiteEUIWN(kBtu/sf)|SourceEUI(kBtu/sf)|SourceEUIWN(kBtu/sf)|SiteEnergyUse(kBtu)|SiteEnergyUseWN(kBtu)|SteamUse(kBtu)|Electricity(kWh)|Electricity(kBtu)|NaturalGas(th

In [64]:
df.dtypes

[('OSEBuildingID', 'int'),
 ('DataYear', 'int'),
 ('BuildingType', 'string'),
 ('PrimaryPropertyType', 'string'),
 ('PropertyName', 'string'),
 ('Address', 'string'),
 ('City', 'string'),
 ('State', 'string'),
 ('ZipCode', 'int'),
 ('TaxParcelIdentificationNumber', 'string'),
 ('CouncilDistrictCode', 'int'),
 ('Neighborhood', 'string'),
 ('Latitude', 'double'),
 ('Longitude', 'double'),
 ('YearBuilt', 'int'),
 ('NumberofBuildings', 'int'),
 ('NumberofFloors', 'int'),
 ('PropertyGFATotal', 'int'),
 ('PropertyGFAParking', 'int'),
 ('PropertyGFABuilding(s)', 'int'),
 ('ListOfAllPropertyUseTypes', 'string'),
 ('LargestPropertyUseType', 'string'),
 ('LargestPropertyUseTypeGFA', 'int'),
 ('SecondLargestPropertyUseType', 'string'),
 ('SecondLargestPropertyUseTypeGFA', 'double'),
 ('ThirdLargestPropertyUseType', 'string'),
 ('ThirdLargestPropertyUseTypeGFA', 'double'),
 ('YearsENERGYSTARCertified', 'double'),
 ('ENERGYSTARScore', 'string'),
 ('SiteEUI(kBtu/sf)', 'double'),
 ('SiteEUIWN(kBtu/sf

In [8]:
df.describe().show()

+-------+------------------+--------+-----------------+-------------------+-----------------+--------------------+-------+-----+-----------------+-----------------------------+-------------------+------------+-------------------+-------------------+-----------------+------------------+-----------------+------------------+------------------+----------------------+-------------------------+----------------------+-------------------------+----------------------------+-------------------------------+---------------------------+------------------------------+------------------------+-----------------+------------------+------------------+------------------+--------------------+--------------------+---------------------+-----------------+------------------+--------------------+------------------+------------------+--------+----------------+------------+------------------+---------------------+
|summary|     OSEBuildingID|DataYear|     BuildingType|PrimaryPropertyType|     PropertyName|      

In [10]:
# number of columns
len(df.columns)

46

In [ ]:
# count dtypes
print(Counter((x[1] for x in df.dtypes)))

Counter({'double': 18, 'string': 16, 'int': 11, 'boolean': 1})


In [52]:
len(numeric_features)

29

In [32]:
# drop non numerical features
df = df[numeric_features]

In [44]:
df.count()

3376

In [11]:
# impute missing values
df = df.fillna(0)

In [13]:
numeric_features

['OSEBuildingID',
 'DataYear',
 'ZipCode',
 'CouncilDistrictCode',
 'Latitude',
 'Longitude',
 'YearBuilt',
 'NumberofBuildings',
 'NumberofFloors',
 'PropertyGFATotal',
 'PropertyGFAParking',
 'PropertyGFABuilding(s)',
 'LargestPropertyUseTypeGFA',
 'SecondLargestPropertyUseTypeGFA',
 'ThirdLargestPropertyUseTypeGFA',
 'YearsENERGYSTARCertified',
 'SiteEUI(kBtu/sf)',
 'SiteEUIWN(kBtu/sf)',
 'SourceEUI(kBtu/sf)',
 'SourceEUIWN(kBtu/sf)',
 'SiteEnergyUse(kBtu)',
 'SiteEnergyUseWN(kBtu)',
 'SteamUse(kBtu)',
 'Electricity(kWh)',
 'Electricity(kBtu)',
 'NaturalGas(therms)',
 'NaturalGas(kBtu)',
 'TotalGHGEmissions',
 'GHGEmissionsIntensity']

In [14]:
import re

In [31]:
# exclude energy targets from features
targets = [
    c for c in numeric_features if bool(re.search("kBtu|kWh|\\(therms|Emissions", c))
]
features = [
    c
    for c in numeric_features
    if not bool(re.search("kBtu|kWh|\\(therms|Emissions", c))
]

In [32]:
# assemble all numerical columns in a vector
vectorAssembler = VectorAssembler(inputCols=features, outputCol="features")
v_df = vectorAssembler.transform(df)
# select features vector and target
v_df = v_df.select(["features", "SiteEnergyUseWN(kBtu)"])
# v_df.show(3)

In [37]:
v_df.show()

+--------------------+---------------------+
|            features|SiteEnergyUseWN(kBtu)|
+--------------------+---------------------+
|[1.0,2016.0,98101...|            7456910.0|
|[2.0,2016.0,98101...|            8664479.0|
|[3.0,2016.0,98101...|          7.3937112E7|
|[5.0,2016.0,98101...|            6946800.5|
|[8.0,2016.0,98121...|          1.4656503E7|
|[9.0,2016.0,98101...|          1.2581712E7|
|[10.0,2016.0,9810...|            6062767.5|
|[11.0,2016.0,9810...|            7067881.5|
|[12.0,2016.0,9810...|          1.4194054E7|
|[13.0,2016.0,9810...|            4807679.5|
|[15.0,2016.0,9810...|           1.664693E7|
|[16.0,2016.0,9810...|          2.7070114E7|
|[17.0,2016.0,9810...|            6358719.5|
|[18.0,2016.0,9810...|          2.2524948E7|
|[19.0,2016.0,9810...|          1.1179203E7|
|[21.0,2016.0,9815...|          1.8706912E7|
|[22.0,2016.0,9810...|          1.0192124E7|
|[23.0,2016.0,9810...|           3.435192E7|
|[24.0,2016.0,9810...|            7877219.0|
|[25.0,201

In [49]:
train_df, test_df = v_df.randomSplit([0.7, 0.3])

In [50]:
train_df.count()

2435

In [51]:
test_df.count()

941

In [52]:
# train linear regression model
lr = LinearRegression(
    featuresCol="features",
    labelCol="SiteEnergyUseWN(kBtu)",
    maxIter=10,
    regParam=0.3,
    elasticNetParam=0.8,
)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [-151.07954359371521,0.0,-53.51730378157163,-240554.4896691815,8212322.653426656,10486418.26409136,-673.0120424320967,-1653700.1162544375,217778.52265853234,-2.16218422315188,-84.81031719628625,-0.0,33.23002323360613,139.22501191859666,407.55695809713234,-1.3963113958138527e-52]
Intercept: 904104916.2293359


In [53]:
# training score
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 12917246.730128
r2: 0.455302


In [54]:
train_df.describe().show()

+-------+---------------------+
|summary|SiteEnergyUseWN(kBtu)|
+-------+---------------------+
|  count|                 2435|
|   mean|    5398252.014250308|
| stddev| 1.7505775193069838E7|
|    min|                  0.0|
|    max|         4.71613856E8|
+-------+---------------------+



In [57]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction", "SiteEnergyUseWN(kBtu)", "features").show(5)

+--------------------+---------------------+--------------------+
|          prediction|SiteEnergyUseWN(kBtu)|            features|
+--------------------+---------------------+--------------------+
| 1.764028689962101E7|          7.3937112E7|[3.0,2016.0,98101...|
|  2360176.4152789116|          1.2581712E7|[9.0,2016.0,98101...|
|   7184788.142513275|           1.664693E7|[15.0,2016.0,9810...|
|2.1202014952000618E7|          4.7023088E7|[26.0,2016.0,9810...|
|1.9397328736707926E7|          4.9539212E7|[27.0,2016.0,9810...|
+--------------------+---------------------+--------------------+
only showing top 5 rows



In [58]:
from pyspark.ml.evaluation import RegressionEvaluator

lr_evaluator = RegressionEvaluator(
    predictionCol="prediction", labelCol="SiteEnergyUseWN(kBtu)", metricName="r2"
)
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

R Squared (R2) on test data = 0.116433


In [59]:
test_result = lr_model.evaluate(test_df)
print(
    "Root Mean Squared Error (RMSE) on test data = %g"
    % test_result.rootMeanSquaredError
)

Root Mean Squared Error (RMSE) on test data = 1.01625e+07


In [60]:
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()

numIterations: 10
objectiveHistory: [0.4999999999999999, 0.4513405332379861, 0.3496288720705346, 0.3132059678991644, 0.2979175219459393, 0.29099582620176784, 0.28634347984746333, 0.2816542883456642, 0.278035308123612, 0.275071666865442, 0.27234891682937795]
+--------------------+
|           residuals|
+--------------------+
| -243489.84001803398|
| -1341477.7421656847|
|  494338.07420897484|
|   515201.9758192301|
| -1263634.9673179388|
| -283987.53382360935|
|  3499325.4105563164|
| -436322.10798954964|
|1.4048734492967248E7|
|  37952.919716358185|
|   738110.4384701252|
|  3012087.1990534067|
|  362873.19047009945|
| -1783005.3338081837|
|1.0624193788422823E7|
|  -2121161.340616703|
|   2397479.845118284|
|  -4644345.676080108|
|  -3645164.687152028|
|   -7219776.80274415|
+--------------------+
only showing top 20 rows

